# 测试集总体准确率评估指标

分析测试集预测结果表格，计算总体准确率评估指标和各类别准确率评估指标。


## 导入工具包

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## 载入类别名称和ID

In [2]:
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

['Angular', 'Rounded', 'Subangular', 'Subrounded', 'Very angular', 'Well rounded']


## 载入测试集预测结果表格

In [3]:
df = pd.read_csv('测试集预测结果.csv')

In [4]:
df

,Image path,Labeling categories ID,Labeling categories name,top-1-Predictions ID,top-1-Predictions name,top-2-Predictions ID,top-2-Predictions name,top-3-Predictions ID,top-3-Predictions name,top-n Predictions correction,Angular-Predictions Confidence Level,Rounded-Predictions Confidence Level,Subangular-Predictions Confidence Level,Subrounded-Predictions Confidence Level,Very angular-Predictions Confidence Level,Well rounded-Predictions Confidence Level
0,Particle Figures_split/val/Angular/Angular (10...,0,Angular,0,Angular,4,Very angular,2,Subangular,True,0.574018,0.003499,0.111645,0.016418,0.294030,0.000390
1,Particle Figures_split/val/Angular/Angular (10...,0,Angular,3,Subrounded,2,Subangular,0,Angular,True,0.020437,0.010734,0.284462,0.672124,0.012037,0.000206
2,Particle Figures_split/val/Angular/Angular (11...,0,Angular,0,Angular,2,Subangular,4,Very angular,True,0.510571,0.087094,0.166346,0.104690,0.122172,0.009128
3,Particle Figures_split/val/Angular/Angular (13...,0,Angular,0,Angular,2,Subangular,4,Very angular,True,0.425345,0.004741,0.424306,0.027567,0.117619,0.000422
4,Particle Figures_split/val/Angular/Angular (13...,0,Angular,0,Angular,2,Subangular,4,Very angular,True,0.682034,0.010761,0.177304,0.050383,0.074711,0.004807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,Particle Figures_split/val/Well rounded/Well r...,5,Well rounded,3,Subrounded,1,Rounded,2,Subangular,False,0.031882,0.301592,0.233697,0.364321,0.050351,0.018157
244,Particle Figures_split/val/Well rounded/Well r...,5,Well rounded,5,Well rounded,3,Subrounded,2,Subangular,True,0.062679,0.092158,0.135548,0.279299,0.009975,0.420340
245,Particle Figures_split/val/Well rounded/Well r...,5,Well rounded,2,Subangular,3,Subrounded,4,Very angular,False,0.037356,0.034042,0.700843,0.175994,0.043497,0.008268
246,Particle Figures_split/val/Well rounded/Well r...,5,Well rounded,2,Subangular,3,Subrounded,1,Rounded,False,0.077578,0.117852,0.449204,0.235666,0.090628,0.029071


## 准确率

In [6]:
sum(df['Labeling categories name'] == df['top-1-Predictions name']) / len(df)

0.5080645161290323

## top-n准确率 #咱们主要看这个 正确率已经非常高了

In [8]:
sum(df['top-n Predictions correction']) / len(df)

0.8911290322580645

## 各类别其它评估指标

In [9]:
from sklearn.metrics import classification_report

In [10]:
print(classification_report(df['Labeling categories name'], df['top-1-Predictions name'], target_names=classes))

              precision    recall  f1-score   support

     Angular       0.44      0.49      0.46        37
     Rounded       0.45      0.32      0.38        31
  Subangular       0.55      0.53      0.54        66
  Subrounded       0.52      0.67      0.59        69
Very angular       0.52      0.39      0.44        31
Well rounded       0.50      0.36      0.42        14

    accuracy                           0.51       248
   macro avg       0.50      0.46      0.47       248
weighted avg       0.51      0.51      0.50       248



macro avg 宏平均：直接将每一类的评估指标求和取平均（算数平均值）

weighted avg 加权平均：按样本数量（support）加权计算评估指标的平均值

In [11]:
report = classification_report(df['Labeling categories name'], df['top-1-Predictions name'], target_names=classes, output_dict=True)
del report['accuracy']
df_report = pd.DataFrame(report).transpose()

In [12]:
df_report

,precision,recall,f1-score,support
Angular,0.439024,0.486486,0.461538,37.0
Rounded,0.454545,0.322581,0.377358,31.0
Subangular,0.546875,0.530303,0.538462,66.0
Subrounded,0.522727,0.666667,0.585987,69.0
Very angular,0.521739,0.387097,0.444444,31.0
Well rounded,0.500000,0.357143,0.416667,14.0
macro avg,0.497485,0.458379,0.470743,248.0
weighted avg,0.506737,0.508065,0.501442,248.0


## 补充：各类别准确率（其实就是recall）

In [13]:
accuracy_list = []
for fruit in tqdm(classes):
    df_temp = df[df['Labeling categories name']==fruit]
    accuracy = sum(df_temp['Labeling categories name'] == df_temp['top-1-Predictions name']) / len(df_temp)
    accuracy_list.append(accuracy)

100%|██████████| 6/6 [00:00<00:00, 715.10it/s]


In [14]:
# 计算 宏平均准确率 和 加权平均准确率
acc_macro = np.mean(accuracy_list)
acc_weighted = sum(accuracy_list * df_report.iloc[:-2]['support'] / len(df))

accuracy_list.append(acc_macro)
accuracy_list.append(acc_weighted)

df_report['accuracy'] = accuracy_list

In [15]:
df_report

,precision,recall,f1-score,support,accuracy
Angular,0.439024,0.486486,0.461538,37.0,0.486486
Rounded,0.454545,0.322581,0.377358,31.0,0.322581
Subangular,0.546875,0.530303,0.538462,66.0,0.530303
Subrounded,0.522727,0.666667,0.585987,69.0,0.666667
Very angular,0.521739,0.387097,0.444444,31.0,0.387097
Well rounded,0.500000,0.357143,0.416667,14.0,0.357143
macro avg,0.497485,0.458379,0.470743,248.0,0.458379
weighted avg,0.506737,0.508065,0.501442,248.0,0.508065


In [16]:
df_report.to_csv('各类别准确率评估指标.csv', index_label='类别')